In [ ]:
from Organizer import Organizer
from search_agent import Research_Tool
import random
LENGTHY_SEARCH_PROMPT = """
                            Answer this question in this format:
                            {"status":"pending", "urls": list(str)}
                            YOU MUST PUT THE JSON ANSWER WITHIN TWO ```

                            Return a list of upto 5 urls from the provided ones that you would like to visit for more Context to answer the following question.
                            Question:
                            
                        """


Researcher = Organizer()
search = Research_Tool()

query = "Tesla by elon musk"
urls = search.search(query, random.randint(10,15))
print(urls)
data = Researcher.get_filtered_urls(urls, LENGTHY_SEARCH_PROMPT, query)
print(data)
processed_data = await Researcher.process_url(data)
final_report = await Researcher.multi_text_processor(query)


final_report

In [ ]:
from bs4 import BeautifulSoup
import jsbeautifier
from urllib.parse import urlparse, urljoin, quote_plus
from typing import Dict, Any, List
import requests


DEBUG = True

def log_debug(message):
    if DEBUG:
        print(f"DEBUG: {message}")

headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }


def _perform_web_search(query: str) -> List[Dict[str, Any]]:
    encoded_query = quote_plus(query)
    search_url = f"https://www.google.com/search?q={encoded_query}&tbm=vid"
    log_debug(f"Search URL: {search_url}")
    
    try:
        log_debug("Sending GET request to Google")
        response = requests.get(search_url, headers=headers, timeout=5)
        log_debug(f"Response status code: {response.status_code}")
        response.raise_for_status()
        
        log_debug("Parsing HTML with BeautifulSoup")
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup)
        # log_debug("Searching for result divs")
        # search_results = []
        # for g in soup.find_all('div', class_='g'):
        #     log_debug("Processing a search result div")
        #     anchor = g.find('a')
        #     title = g.find('h3').text if g.find('h3') else 'No title'
        #     url = anchor.get('href', 'No URL') if anchor else 'No URL'
            
        #     description = ''
        #     description_div = g.find('div', class_=['VwiC3b', 'yXK7lf'])
        #     if description_div:
        #         description = description_div.get_text(strip=True)
        #     else:
        #         description = g.get_text(strip=True)
            
        #     log_debug(f"Found result: Title: {title[:30]}..., URL: {url[:30]}...")
        #     search_results.append({
        #         'title': title,
        #         'description': description,
        #         'url': url
        #     })
        
        # log_debug(f"Successfully retrieved {len(search_results)} search results for query: {query}")
        # return search_results
    except requests.RequestException as e:
        log_debug(f"Error performing search: {str(e)}")
        return []

_perform_web_search("robovan")

In [ ]:
print(final_report)

In [ ]:
import requests
from utils import split_corpus
import concurrent.futures
import openai
import json
import re

query = """Generate me a report within 500 words in JSON format using this schema:
            {"summery": list({"heading": sescription}, ...), "Images": list({"url": str, "description": str}, ...), "links": list({"url": str, "description": str}, ...)}
            The Images and the Links Must be Present in the context. if no Image or Link found keep the list empty.
            DO NOT USE YOUR KNOWLEDGE TO CONSTRUCT THE ANSWER. USE ONLY WHAT IS PROVIDED IN THE CONTEXT.
            add only the image and links that are strongly related to the report topic.
            in the summery describe the Report topic based on the provided context briefly within 800 words.
            Report Topic:
            Elon Musk"""
res = requests.get("https://en.wikipedia.org/wiki/Elon_Musk")

data = split_corpus(res.text, max_words=1350)
api_keys = open("keys.txt").read().split("\n")

print(len(data))

def extract_query(text: str) -> str:
    pattern = r"```(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0] if matches else text

def ask_llm(data):
    query, api_key, model, JSON = data
    for i in range(5):
        SambaNova_Client = openai.OpenAI(
                api_key=api_key,
                base_url="https://api.sambanova.ai/v1",
            )

        response = SambaNova_Client.chat.completions.create(
            model=model,
            messages=[{"role":"system","content": "You're a advance data analyst."},{"role":"user","content":query}],
            temperature =  0.1,
            top_p = 0.1,
        )
        if not JSON:
            # print(model)
            return response.choices[0].message.content
        try:
            data = json.loads(extract_query(response.choices[0].message.content))
            return data
        except Exception as e:
            print(e)
            print(response.choices[0].message.content)

final_data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=len(api_keys)) as executor:
        prep = [(i+query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", True ) for m,i in enumerate(data)]
        futures = {executor.submit(ask_llm, task): task for task in prep}

        for future in concurrent.futures.as_completed(futures):
            task = futures[future]
            try:
                result = future.result()
                print(result)
                final_data.append(result)
            except Exception as e:
                print(f"Task {task} raised an exception: {e}")


In [10]:
x = split_corpus(str(final_data), max_words=1350)
print(len(x))
# x_data = []
# for m,i in enumerate(x):
#     temp =(i+query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", True )
#     res = ask_llm(temp)
#     x_data.append(res)
# x_data


1


In [11]:
y = split_corpus(str(final_data), max_words=1350)
print(len(y))
query = """
                            {data}
                            You have been provided with a report and some images and links related to this question:
                            {question}

                            your job is to unify them and create a Final report that consists of all three of these in MARKDOWN format.
                            Take the images and the link urls from the provided urls only and don't make them up yourself.

                            NOTE: PUT THE IMAGE URLS IN A WAY SO THAT THEY CAN BE DISPLAYED DIRETLY ON THE MARKDOWN.
                                  PUT THE REFERENCE AS YOU SEE FIT.
                                  MAKE THE REPORT LOOK AS HUMAN LIKE AS POSSIBLE.
                            """.format(data = y[0], question = "Jeff Bezos")
y_data = []
for m,i in enumerate(y):
    temp =(query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", False )
    res = ask_llm(temp)
    y_data.append(res)

print(y_data[0])

1


# Report: Tesla's Plans for a Van and the Robovan

## Introduction

Tesla has been hinting at developing a van, with mentions of "high passenger-density urban transport" and autonomous buses in its Master Plan Part Deux. Despite Elon Musk's disdain for public transportation, he has listed "bus" and "commercial/passenger vans" as "TBD" in Tesla's Master Plan Part 3.

## Electric Van Market

The electric van market is heating up, with recent releases from Volkswagen, Mercedes, and others, which may prompt Tesla to enter the market.

## Tesla's Robovan

Tesla's Robovan is the surprise of the night, a new van that Elon Musk has dropped hints about in the past. The Robovan has a sleek train engine-like shape and no visible wheels, and can carry up to 20 people or be used to transport goods.

![Tesla Robovan](https://duet-cdn.vox-cdn.com/thumbor/0x0:5662x3184/2400x1600/filters:focal(2831x1592:2832x1593):format(webp)/cdn.vox-cdn.com/uploads/chorus_asset/file/25670518/Robovan_73.jpg)

## Purpo

In [8]:
from utils import fetch_videos
from utils import perform_image_search

img = perform_image_search("anime")
img

[{'image_url': 'https://sm.ign.com/ign_pk/lists/t/the-top-25/the-top-25-greatest-anime-characters-of-all-time_xkdu.jpg',
  'description': 'The Top 25 Greatest Anime Characters of All Time',
  'page_url': 'https://pk.ign.com/afro-samurai/187959/feature/the-top-25-greatest-anime-characters-of-all-time'},
 {'image_url': 'https://m.media-amazon.com/images/I/619RmKdoIiL._AC_UF1000,1000_QL80_.jpg',
  'description': 'Anime Notebook: anime cat girl notebook for girl otaku | Anime girl | gift  for girle wide ruled notebook | anime cat girl style',
  'page_url': 'https://www.amazon.com/Anime-Notebook-anime-notebook-otaku/dp/B09CSKMRCJ'},
 {'image_url': 'https://sm.ign.com/t/ign_nordic/lists/t/the-15-bes/the-15-best-shonen-anime-series-of-all-time_jnrw.1200.jpg',
  'description': 'The 15 Best Shonen Anime Series of All Time',
  'page_url': 'https://nordic.ign.com/attack-on-titan-1/73391/lists/the-15-best-shonen-anime-series-of-all-time'},
 {'image_url': 'https://i.ytimg.com/vi/xXmXM0qRMbo/hq720.j

In [12]:
fetch_videos("anime", 10)

[{'content': 'https://www.youtube.com/watch?v=rNZBVNNmRsc',
  'description': 'Anime series are one of the best fantasy, sci-fi , with unique stories but the language is not available in all languages .So we creates this Anime Tamil voice channel and tries to make the language is not to be a hurdles for the anime lovers and the people like more fantasy , adventure, comedy, sci-fi, with new unique stories ...',
  'duration': '8:13',
  'embed_html': '<iframe width="1280" height="720" src="https://www.youtube.com/embed/rNZBVNNmRsc?autoplay=1" frameborder="0" allowfullscreen></iframe>',
  'embed_url': 'https://www.youtube.com/embed/rNZBVNNmRsc?autoplay=1',
  'image_token': 'f4e6aebfdc759ee6b2debe36cbd018644e2e1f4439b43284c739613254fffa05',
  'images': {'large': 'https://tse4.mm.bing.net/th?id=OVF.yuvSDIheBKlrRyXfcAaonA&pid=Api',
   'medium': 'https://tse4.mm.bing.net/th?id=OVF.yuvSDIheBKlrRyXfcAaonA&pid=Api',
   'motion': '',
   'small': 'https://tse4.mm.bing.net/th?id=OVF.yuvSDIheBKlrRyXfc

In [ ]:
https://tse4.mm.bing.net/th?id=OVF.yuvSDIheBKlrRyXfcAaonA&pid=Api